<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

## Bewertung binärer Klassifikatoren

In diesem Notebook wollen wir betrachten, wie die man die Güte von binären Klassifikatoren bewerten kann.
Als Beispiel dient uns der [Heart Failure Prediction](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction) Datensatz von [Kaggle](www.kaggle.com).
In diesem Datensatz wurden 5 Herzdatensätze aus verschiedenen Ländern über 11 gemeinsame Merkmale kombiniert.
Diese Merkmale stellen medizinischen Werte dar, die Zielvariable sagt aus, ob die entsprechende Person herzkrank ist.

Die folgende Liste beschreibt die Merkmale und die Zielvariable des Datensatzes:<br>
**Age**: Alter des Patienten [Jahre]<br>
**Sex**: Geschlecht des Patienten [M: Männlich, F: Weiblich]<br>
**ChestPainType**: Art der Brustschmerzen [TA: Typische Angina, ATA: Atypische Angina, NAP: Nicht-Anginaler Schmerz, ASY: Asymptomatisch]<br>
**RestingBP**: Ruheblutdruck [mm Hg]<br>
**Cholesterin**: Serumcholesterin [mm/dl]<br>
**FastingBS**: Nüchtern-Blutzucker [1: wenn FastingBS > 120 mg/dl, 0: sonst]<br>
**RestingECG**: Ruhe-Elektrokardiogramm-Ergebnisse [Normal: Normal, ST: mit ST-T-Wellen-Anomalie, LVH: mit Hypertrophie]<br>
**MaxHR**: maximal erreichte Herzfrequenz [Numerischer Wert zwischen 60 und 202]<br>
**ExerciseAngina**: Belastungsangina [J: Ja, N: Nein]<br>
**Oldpeak**: ST-Senkung = ST [Numerischer Wert]<br>
**ST_Slope**: die Steigung des Spitzen-ST-Segments bei Belastung [Up: ansteigend, Flat: flach, Down: absteigend]<br>
**HeartDisease**: Ausgabeklasse [1: Herzkrankheit, 0: Normal]<br>


Wir starten mit dem Herunterladen des Datensatzes und dem Aufteilen in Trainings- und Testdaten.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import urllib.request
import os

In [ ]:
url = "https://github.com/fhswf/datasets/raw/main/heart_imb.csv"
dfile = "./heart_imb.csv"

if not os.path.isfile(dfile):
    urllib.request.urlretrieve(url, dfile)

In [ ]:
df = pd.read_csv(dfile)
X, y = df.drop(columns=["HeartDisease"]), df["HeartDisease"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
distrib = y_train.value_counts().values
print(f"In den Traingsdaten sind {distrib[0]} gesunde und {distrib[1]} herzkranke Personen")
distrib = y_test.value_counts().values
print(f"Im Testdatensatz sund {distrib[0]} gesunde und {distrib[1]} herzkranke Personen")

Als Grundlage für unsere Bewertungen trainieren wir zunächst ein `DummyClassifier` Modell.
Das ist kein Klassifikator im eigentlichen Sinn, denn der `DummyClassifier` verwendet keines der Merkmale zur Vorhersage der Zielariablen. Das Modell schaut sich lediglich die Verteilung der Labels an und schätzt dann anhand sehr einfacher Regeln. In unserem Fall verwenden wir die Regel `most_frequent`, was bedeutet, dass immer die am häufigsten vorkommende Klasse vorausgesagt wird. Dies ist bei uns die Klasse 0, also die Klasse für *gesunde Personen*.

Der *DummyClassifier* schneidet für unsere Daten mit 90% *Treffergenauigkeit* (***accuracy***) sehr gut ab. Überlegen Sie sich, warum das bei unseren Daten der Fall ist.

In [ ]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train, y_train)
acc_dummy = dummy.score(X_test, y_test)
print(f"Score (Dummy): {acc_dummy:.3f}")

Nun wollen wir ein *echtes* Modell trainieren und verwenden hier die Logistische Regression.
Da unser Datensatz numerische und kategorische Merkmale enthält, müssen wir ihn noch Transformieren.
Damit die Merkmale in etwa proportional zueinander gewichtet werden, müssen die Spalten noch normalisiert bzw. standardisiert werden.

Außerdem sollten im Vorfeld fehlende Werte ersetzt oder die zugehörigen Datenpunkte gelöscht werden. Da unser Datensatz vollständig ist, benötigen wir diesen Schritt hier eigentlich nicht, nehmen ihn aber zur Vollständigkeit mit auf.

Die Transformationsschritte müssen beim Training und der Inferenz durchgeführt. Das führt zu recht vielen Aufrufen und damit tendenziell zu eher unübersichtlichem Code.
Daher verwenden wir hier eine *sklearn Pipeline* die mehrere Vorverarbeitungsschritte sowie das Modell in einem Objekt zusammenfasst.

Bei **numerischen Merkmalen** werden zunächst fehlende Werte durch den Mittelwert ersetzt, danach werden die Spaltewerte standardisiert.
Die **kategorischen Merkmale** werden per One-Hot-Coding umgewandelt.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


numeric_features = X_train.select_dtypes(exclude="object").columns.values
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = X_train.select_dtypes("object").columns.values
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

Der für die Vorverarbeitung zuständige `ColumnTransformer` wird nun mit einem `LogisticRegression` Modell in einer *Pipeline* zusammengefasst.

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

**Aufgabe:** Verwenden Sie die Pipeline `logreg`, um die *Accuracy* für die Testdaten zu bestimmen.

In [ ]:
acc_logreg = None
# YOUR CODE HERE
raise NotImplementedError()
print(f"Score (Log Regression): {acc_logreg:.3f}")

In [ ]:
from numpy.testing import assert_almost_equal
assert_almost_equal(acc_logreg, 0.9217391304347826, decimal=4)

Auf unsere Daten angewendet, erzielt das Modell eine *Treffergenauigkeit* (***accuracy***) von 92.2%, ist also nur geringfügig besser als unser *DummyClassifier*. Bedeutet dies, dass unser Modell schlecht ist?

#### Konfusionsmatrix 

Die Konfusionsmatrix (engl. *Confusion Matrix*) ist ein wichtiges Hilfsmittel zur Bewertung von Klassifikatoren.
Auf der x-Achse Confusion Matrix sind üblicherweise die vorhergesagten Klassen aufgetragen, auf der y-Achse die tatsächlichen Klassen. Bei einer binären Klassifikation besitzt die Matrix also 4 Zellen, bzw. Einträge.

Auf der obere Zeile der binären Confusion Matrix sind die Datenpukte aufgetragen, die *tatsächlich* das Merkmal `0` besitzen (`y_test==0`), also hier die *Gesunden*. Auf der zweiten Zeile stehen die Datenpunkte, die *Herzkranken* zugeordnet sind (`y_test==1`).

In der ersten Spalte der Confusion Matrix sind die Datenpukte aufgetragen, die vom Klassifikator das Ergebnis `0` zugeordnet bekommen (`prediction==0`), auf der zweiten Zeile diejenigen, für die das Modell eine `1` vorhersagt (`prediction==1`).

Die Zellen oben links und unten rechts sind also richtig klassifizierte, in den Zellen oben rechts und unten links stehen falsch klassifizierte Datenpunkte.
Für die 4 Zellen der Matrix haben sich folgende englische Begriffe auch im Deutschen etabliert:
- Richtig als positiv klassifiziert: *true positives* (**TP**)
- Fälschlicherweise als positiv klassifiziert: *false positives* (**FP**)
- Richtig als negativ klassifiziert: *true negatives* (**TN**)
- Fälschlicherweise als negativ klassifiziert: *false negatives* (**FN**)

Die *Accuracy* stellt die richtig klassifzierten in Relation zu allen Datenpunkten.
Das ist aber häufig nicht das Kriterium, das zur Bewertung des Klassifikators sinnvoll ist.

Betrachten wir unseren Fall mit dem Prediktor für Herzkrankheiten.
Das Modell sollte möglichst alle Personen, die tatsächlich herzkrank sind auch als krank erkennen.
Der Fehler, dass eine gesunde Person fälschlicherweise als herzkrank klassifiziert wird, ist eher zu akzeptieren.

Schauen wir uns die Confusion Matrix mit *sklearn* an:

In [ ]:
import sklearn
from packaging import version
if version.parse(sklearn.__version__) > version.parse("0.24"):
    from sklearn.metrics import ConfusionMatrixDisplay
    cm_function = ConfusionMatrixDisplay.from_estimator
else:
    from sklearn.metrics import plot_confusion_matrix
    cm_function = plot_confusion_matrix
    
cm_function(
    logreg, X_test, y_test,
    display_labels=["Gesund", "Herzkrank"],
    values_format="d", cmap='Blues');

So betrachtet, sieht unser Klassifikator nicht mehr so gut aus.
Von den insgesamt 23 kranken Personen werden immerhin 7 als gesund klassifiziert.

Das Verhältnis von richtig als positiv klassifizierten Datenpunkten zu allen tatsächlich positiven, bezeichnet man als Sensitivität (*senitivity* oder auch ***recall***).
Das Verhältnis von richtig als positiv klassifizierten Datenpunkten zu allen als positiven klassifizierten Datenpunkten bezeichnet man als Relevanz (***precision***). 
Ein Maß, das beide Kriterien gleich gewichtet miteinbezieht, ist der F1-Score.

In [ ]:
from sklearn.metrics import confusion_matrix
pred = logreg.predict(X_test)
TN, FP, FN, TP = confusion_matrix(y_test, pred).ravel()
TN, FP, FN, TP

**Aufgabe:** Berechnen Sie jeweils den **recall**, die **precision** und den **F1-Score** für das Modell. Verwenden Sie dazu die Variablen `TN`, `FP`, `FN` und `TP`, wie oben berechnet.

In [ ]:
recall = None
# YOUR CODE HERE
raise NotImplementedError()
print("TP = %d, FN = %d" % (TP, FN))
print("Recall: %0.4f" % (recall))

In [ ]:
from numpy.testing import assert_almost_equal
assert_almost_equal(recall, 0.6956521739130435, decimal=4)

In [ ]:
precision = None
# YOUR CODE HERE
raise NotImplementedError()
print("TP = %d, FP = %d" % (TP, FP))
print("Precision: %0.4f" % (precision))

In [ ]:
from numpy.testing import assert_almost_equal
assert_almost_equal(precision, 0.5925925925925926, decimal=4)

In [ ]:
f1 = None
# YOUR CODE HERE
raise NotImplementedError()
print("f1-score: %0.4f" % (f1))

In [ ]:
from numpy.testing import assert_almost_equal
assert_almost_equal(f1, 0.64, decimal=4)

Da man diese und weitere Metriken recht häufig benötigt, besitzt *sklearn* eigene Methoden zur Berechnug im Modul `metrics`. 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def print_scores(predictions, y_test):
    acc = accuracy_score(y_test, predictions)
    pre = precision_score(y_test, predictions)
    rec = recall_score(y_test, predictions)
    f1s = f1_score(y_test, predictions)
    print(f"accuracy={acc:.3f} precision={pre:.3f} recall={rec:.3f} f1={f1:.3f}")

print_scores(logreg.predict(X_test), y_test)

#### Sensitivität verbessern

Wir haben erörtert, dass für unsere Anwendung die Sensitivität das wichtigste Kriterium ist und gerade in diesem Bereich schneidet unser Klassifikator mit einem Recall-Wert von ca. 70% nicht gut ab.
Es stellt sich die Frage, ob und wie wir den *recall*, ggf. auch zu Ungunsten der Relevanz (*precision*) und Treffergenauigkeit (*accuracy*), verbessern können.

Dazu schauen wir etwas genauer in unser Logistisches Regressionsmodell.
Das Modell berechnet im Prinzip für jeden Datenpunkt eine *Wahrscheinlichkeit*, zur Klasse `1` zu gehören.
Am Ende wird dann ein Schwellenwert (engl. *threshold*) verwendet um die binäre Klasse zu bestimmen.
Üblicherweise wird als Schwellenwert 0.5 gewählt.

Die berechneten Wahrscheinlichkeiten können wir für unser `LogisticRegression` Modell mit der Methode `predict_proba` berechnen.
Schauen wir nun an, wo die berechneten Wahrscheinlichkeiten über dem Wert `0.5` liegen und vergleichen wir das Ergebnis mit den tatsächlichen Labels (`y_test`), bekommen wir die gleichen Ergebnisse wie oben:

In [ ]:
pred = logreg.predict_proba(X_test)[:, 1] > .5
print_scores(pred, y_test)

Wenn wir wollen, dass ein Datenpunkt *eher als herzkrank klassifiziert* werden soll, müssen wir den Schwellenwert herabsetzen.
Schauen wir uns also an was passiert, wenn wir den Schwellenwert auf `0.2` ändern:

In [ ]:
pred = logreg.predict_proba(X_test)[:, 1] > .2
print_scores(pred, y_test)

Der Recall hat sich deutlich verbessert.
Dass die Accuracy und die Precision zurückgegangen sind, war erwartet.

Schauen wir uns die Ergebnisse nochmal in einer Confusion Matrix an:

In [ ]:
if version.parse(sklearn.__version__) > version.parse("0.24"):
    ConfusionMatrixDisplay.from_predictions(
        y_test, pred,
        display_labels=["Gesund", "Herzkrank"],
        values_format="d", cmap='Blues')
else:
    print("Bitte sklearn updaten!")
    print("z.B. mit: conda install scikit-learn=1.0.1")

#### Precision-Recall- und ROC-Kurve

Wie sich die Verschiebung des Schwellenwertes auswirkt, kann grafisch mit der *Precision-Recall* (PR) oder der *Receiver Operating Characteristic* (ROC) Kurve dargestellt werden.

Bei der PR_Kurve stellt man auf der x-Achse die Relevanz (*precision*) und auf der y-Achse die Sensitivität (*recall*) dar.
Die Kurve zeigt den Verlauf beider Werte bei steigendem Schwellenwert.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(
    y_test, logreg.predict_proba(X_test)[:, 1]
)
plt.plot(precision, recall, label="PR Kurve")
plt.xlabel("Precision")
plt.ylabel("Recall")
plt.plot(
    precision_score(y_test, logreg.predict(X_test)),
    recall_score(y_test, logreg.predict(X_test)),
    "ob",
    markersize=10,
    label="Schwellenwert 0.5"
)

plt.legend(loc="best");

**Aufgabe:** Die Kurve scheint hier *zurückzuspringen* weil sich *Recal* und *Precision* gleichzeitig verschlechtern. Haben Sie eine Idee, warum das sein kann?

YOUR ANSWER HERE

Die ROC-Kurve betrachtet auf der y-Achse ebenfalls den *Recall* und auf der x-Achse die ***Ausfallrate*** (engl. *false positive rate*, FPR).

Da der *Recall* auch als *true positive rate* angesehen werden kann, werden bei der ROC Kurve also die *richtig als positiv Klassifizierten* den *falsch als positiv Klassifizierten* gegenübergestellt.

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, logreg.predict_proba(X_test)[:, 1])
plt.plot(fpr, tpr, label="ROC Kurve")
plt.xlabel("FPR")
plt.ylabel("TPR (recall)")

pred = logreg.predict(X_test)
TN, FP, FN, TP = confusion_matrix(y_test, pred).ravel()

plt.plot(
    FP/(FP+TN),
    TP/(TP+FN),
    "ob",
    markersize=10,
    label="Schwellenwert 0.5"
)
plt.legend(loc="best");

**Aufgabe:** Die ROC-Kurve steigt hier monoton an. Ist das immer der Fall?

YOUR ANSWER HERE

#### Klassifikation bei unausgewogenen Datensätzen

Sie haben sicher schon bemerkt, dass ein Problem mit unserem Datensatz darin besteht, dass die Verteilung der Zielvariablen sehr unausgeglichen ist.
Wir haben fast 10-mal mehr Datenpunkte, die zu *gesunden* Personen ghören.
Damit sind die *Herzkranken* im Datensatz stark unterrepräsentiert.

In [ ]:
distrib = y_train.value_counts().values
print(f"In den Traingsdaten sind {distrib[0]} gesunde und {distrib[1]} herzkranke Personen")

Eine Möglichkeit, mit unausgewogenen Datensätzen umzugehen, ist Under- bzw. Oversamplig.
Beim **Oversampling** verden Datenpunkte aus der unterrepräsentierten Klasse dupliziert.
**Undersampling** löscht nach dem Zufallsprinzip Datenpunkte aus der überrepräsentierten Klasse.
Da so der Datensatz aktiv *verkleinert* wird, kann Undersampling dazu führen, dass wertvolle Informationen verloren gehen.

Eine Alternative zu Under- bzw. Oversamplig ist, die Fehler beim Modell-Training unterschiedlich zu gewichten.
Falsch klassifizierte *Kranke* könnten stärker gewichtet werden, als falsch klassifizierte *gesunde* Personen.
Dies würde das Modell dazu zwingen, die richtigen Ergebnisse für die Klasse 1 (=*herzkrank*) zu bevorzugen.

Die Klassen-Gewichte können in *sklearn* über den Parameter `class_weights` eingestelt werden.

**Aufgabe:** Erstellen Sie eine *sklearn Pipeline* genau wie im Beispiel oben.
Fügen Sie zu dem `LogisticRegression`-Modell den Parameter `class_weight` hinzu.
Der Wert des des Parameters ist ein *Dictionary*, in dem die Klassen als *Keys* und die zugehörigen Gewichte als *Values* aufgeführt sind.
Wählen Sie für die Klasse `0` das Gewicht `1` und für die Klasse `1` das Gewicht `10`.

*Zur Erinnerung: Das folgende Dictionary `d` ordnet den Keys `4` und `5` die Values `'Hallo'` bzw. `'Welt'` zu*
```python
d = {4: 'Hallo', 5: 'Welt'}
```

In [ ]:
logreg_cw = None
# Legen Sie die Pipeline unter dem Namen logreg_cw an!

# YOUR CODE HERE
raise NotImplementedError()

logreg_cw.fit(X_train, y_train)
print_scores(logreg_cw.predict(X_test), y_test)

In [ ]:
from numpy.testing import assert_almost_equal
__acc = logreg_cw.score(X_test, y_test)
assert_almost_equal(__acc, 0.8478260869565217, decimal=4)

Neben fest angegeben Klassen-Gewichten kann `class_weight` auch auf den Wert `'balanced'` gesetzt werden. In diesem Fall werden die Gewichte aus der Verteilung der Labels berechnet.

**Aufgabe:** Erzeugen Sie eine neue Pipeline mit einem `LogisticRegression`-Modell, wobei der Parameter `class_weight` auf den Wert `'balanced'` gesetzt wird.

In [ ]:
logreg_bal = None
# Legen Sie die Pipeline unter dem Namen logreg_bal an!

# YOUR CODE HERE
raise NotImplementedError()

logreg_bal.fit(X_train, y_train)
print_scores(logreg_bal.predict(X_test), y_test)

In [ ]:
from numpy.testing import assert_almost_equal
__acc = logreg_bal.score(X_test, y_test)
assert_almost_equal(__acc, 0.8565217391304348, decimal=4)

#### Modelle vergleichen

Wir haben nun unterschiedliche Modelle erzeugt und wollen diese miteinander vergleichen.
Dazu können wir die verschiedenen *Scores* (accuracy, precision, recall, etc.) zurate ziehen.
Allerdings gehen diese Werte alle von einem Schwellenwert von 0.5 aus.

Wenn wir wissen wollen, welches Modell *insgesamt* besser ist, müssen wir auch unterschiedliche Schwellenwerte betrachten.
Dafür können wir die ROC-Kurven in ein Diagramm plotten.

Ein Modell ist umso besser, je *eckiger* der Verlauf der Kurve ist und umso mehr die Kurve die anderen ROC-Kurven *umschließt*.
Mathematisch gesehen, ist also die Fläche unter der ROC-Kurve ausschlaggebend für die Güte des Modells hinsichtlich der Sensitivität.
Diese Fläche wird auch als *Area Under Curve* (AUC) bezeichnet und kann mit der Methode `roc_auc_score` aus dem Modul `sklearn.metrics` berechnet werden.


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

fpr0, tpr0, _ = roc_curve(y_test, logreg.predict_proba(X_test)[:, 1])
plt.plot(fpr0, tpr0, label="ROC Original")
fpr1, tpr1, _ = roc_curve(y_test, logreg_cw.predict_proba(X_test)[:, 1])
plt.plot(fpr1, tpr1, label="ROC Class Weight")
fpr2, tpr2, _ = roc_curve(y_test, logreg_bal.predict_proba(X_test)[:, 1])
plt.plot(fpr2, tpr2, label="ROC Balanced")

plt.xlabel("FPR")
plt.ylabel("TPR (recall)")

plt.xlim([-.02, .5])
plt.legend(loc="best", prop={'size': 14});


print(f"AUC Original: {roc_auc_score(y_test, logreg.predict(X_test)):.3f}")
print(f"AUC Class Weight: {roc_auc_score(y_test, logreg_cw.predict(X_test)):.3f}")
print(f"AUC Balanced: {roc_auc_score(y_test, logreg_bal.predict(X_test)):.3f}")